In [27]:
import findspark
findspark.init()
findspark.find()

'H:\\SPARK'

In [28]:
from pyspark.sql import SparkSession

# Initialize SparkSession with necessary configurations
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark') \
    .config("spark.driver.memory", "15g") \
    .config("spark.hadoop.home.dir", "H:/HADOOP/") \
    .config("spark.hadoop.conf.dir", "H:/HADOOP/etc/hadoop/") \
    .getOrCreate()

import sys
sys.path.append("G:\Dissertation_Project")

# Get SparkContext from the SparkSession
sc = spark.sparkContext


In [29]:
spark

### BASE DATASET

In [30]:
base_df = spark.read.csv("../../Data/Custom_Datasets/conversation_datasets_GPT.csv", header=True, inferSchema=True)
base_df.show(10, truncate=False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Conversation_ID|Attacker_Helper                                                                                                                                                 |Victim                                                                                                                                                                                         |Conversation_Type|
+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------

### PREPROCESSED DATASET

In [31]:
preprocessed_df = spark.read.csv("../../Data/Preprocessed_Datasets/GPT_dataset_preprocessed.csv", header=True, inferSchema=True)
preprocessed_df.show(10, truncate=False)

+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Convert Conversation Columns into actual Arrays

In [32]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import ast

# UDF to convert string representation of list to actual list
def str_to_array_of_arrays(s):
    # Convert the string to a list and then wrap it inside another list
    return [ast.literal_eval(s)][0]

str_to_array_of_arrays_udf = udf(str_to_array_of_arrays, ArrayType(ArrayType(StringType())))

df = preprocessed_df.withColumn("Attacker_Helper", str_to_array_of_arrays_udf(preprocessed_df["Attacker_Helper"])).withColumn("Victim", str_to_array_of_arrays_udf(preprocessed_df["Victim"]))

df.printSchema()

root
 |-- Conversation_ID: string (nullable = true)
 |-- Attacker_Helper: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Victim: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Conversation_Type: integer (nullable = true)



### Loading the pipeline and transforming the data

In [33]:
from pyspark.ml import PipelineModel
from src.CustonTransformers import FlattenTransformer

pipeline_model_path = "./Pipelines/TF-IDF_Pipeline"

pipeline = PipelineModel.load(path=pipeline_model_path)

df_assembled = pipeline.transform(df)

### Splitting the Data

In [34]:
(train_data, test_data) = df_assembled.randomSplit([0.8, 0.2], seed=42)
train_data.show(truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Support Vector Machines

## Training Support Vector Machine Model

In [35]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lsvc = LinearSVC(labelCol="Conversation_Type", featuresCol="combined_features", predictionCol="Prediction")

evaluator = MulticlassClassificationEvaluator(labelCol='Conversation_Type', metricName="accuracy", predictionCol="Prediction")


paramGrid = ParamGridBuilder() \
    .addGrid(lsvc.regParam, [0.1, 0.01, 0.001]) \
    .addGrid(lsvc.maxIter, [10, 20, 50, 100]) \
    .build()
    

tvs = TrainValidationSplit(estimator=lsvc,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           parallelism=2,
                           seed=42)

tvsModel = tvs.fit(train_data)

print(tvsModel, '\n')
print(tvsModel.explainParams(), '\n')
print('\nEvaluating with metric -> {}'.format(evaluator.getMetricName()))
print('Evaluators rating for the test set -> {}'.format(evaluator.evaluate(tvsModel.transform(test_data))))


TrainValidationSplitModel_c7f1e0314b75 

estimator: estimator to be cross-validated (current: LinearSVC_4fa45d3f264e)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LinearSVC_4fa45d3f264e', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='LinearSVC_4fa45d3f264e', name='maxIter', doc='max number of iterations (>= 0).'): 10}, {Param(parent='LinearSVC_4fa45d3f264e', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='LinearSVC_4fa45d3f264e', name='maxIter', doc='max number of iterations (>= 0).'): 20}, {Param(parent='LinearSVC_4fa45d3f264e', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='LinearSVC_4fa45d3f264e', name='maxIter', doc='max number of iterations (>= 0).'): 50}, {Param(parent='LinearSVC_4fa45d3f264e', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='LinearSVC_4fa45d3f264e', name='maxIter', doc='max number of iterations (>= 0).'): 100}, {Param(parent

In [36]:
print('Train Validation Split Model validationMetrics -> {}'.format(tvsModel.validationMetrics))
print('Best Model -> {}'.format(tvsModel.bestModel))

Train Validation Split Model validationMetrics -> [0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8, 0.7666666666666667, 0.8, 0.8]
Best Model -> LinearSVCModel: uid=LinearSVC_4fa45d3f264e, numClasses=2, numFeatures=36


### Testing

In [44]:
predictions = tvsModel.bestModel.transform(test_data)
predictions.select("Conversation_ID","Conversation_Type", "Prediction").where(predictions.Prediction != predictions.Conversation_Type).show(10, truncate=False)

+---------------+-----------------+----------+
|Conversation_ID|Conversation_Type|Prediction|
+---------------+-----------------+----------+
|ALtIzDsxTHTxIe |0                |1.0       |
|GIjmhRS9Dy8xg1 |0                |1.0       |
|OHJFQTvA2eXtsV |1                |0.0       |
|j2Q9T9FmncM5Re |0                |1.0       |
|qEKhQ6c7Jibf99 |0                |1.0       |
+---------------+-----------------+----------+



In [45]:
model_path = "../Models/Trained_Models/SupportVectorMachine"

In [46]:
tvsModel.write().overwrite().save(model_path)